In [70]:
import sqlite3
from xml.etree import ElementTree as ET

con = sqlite3.connect('customers.db')

cur = con.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customerNo TEXT PRIMARY KEY,
    credentials_login,
    profile_creationDate,
    profile_firstName,
    profile_subscribedEmail
)
""")
cur.execute("""
CREATE UNIQUE INDEX IF NOT EXISTS customers_credentials_login
ON customers(credentials_login);
"""
)

con.rollback()

In [71]:
CUSTOMERS_FILE = "cust.xml"
NS = "http://www.demandware.com/xml/impex/customer/2006-10-31"


In [72]:
count = 0
for event, customer in ET.iterparse(CUSTOMERS_FILE, tag="{%s}customer" % NS):
    id = customer.attrib["customer-no"]
    # repeat for queries into xml
    credentials_login = customer.find('.//C:credentials/C:login', namespaces={'C': NS}).text
    profile_firstName = customer.find('.//C:profile/C:first-name', namespaces={'C': NS}).text
    profile_creationDate = customer.find('.//C:profile/C:creation-date', namespaces={'C': NS}).text

    custom_attributes = customer.find('.//C:profile/C:custom-attributes', namespaces={'C': NS})
    profile_subscribedEmail = custom_attributes.find(".//C:custom-attribute[@attribute-id='subscribedEmail']", namespaces={'C':NS}).text

    try:
        cur.execute("insert into customers values (?, ?, ?, ?, ?)",
                    (
                        id,
                        credentials_login,
                        profile_firstName,
                        profile_creationDate,
                        profile_subscribedEmail
                    ))
    except sqlite3.IntegrityError as e:
        print(e, id)
        continue
    if count % 10 == 0:
        print("Done %s" % count)
        print("Last: ", id, credentials_login, profile_firstName, profile_creationDate)
    count = count + 1

TypeError: iterparse() got an unexpected keyword argument 'tag'

In [65]:
cur.close()
con.commit()